# Compare TMA balancing data from previous events with the next events

We are getting ready to balance the telescope twice in the next weeks. First, we will balance the telescope with ComCam and M2 Glass. The M2 Glass and M2 Surrogate have similar weights, with a small difference. We expect the torques applied by the elevation drives will be very close to the previous balancing event(s). A couple of weeks later, we will repeat the procedure with ComCam, M2 Glass, and M1M3 Glass. The M1M3 Glass and M1M3 Cell assembly is much heavier than the M1M3 Mass Simulator (yellow cross) and hundreds of kilograms heavier than the M1M3 Surrogate and M1M3 Cell configuration. This procedure will be much more delicate due to the size and mass of the mirror.

We want to establish a baseline before we start the procedure, and we need someone to review the data to determine whether we can proceed quickly.

The links below point to old night logs that might contain useful information. Feel free to unlink them if they are not useful.

Here is an approximate timeline of different integration phases where we needed to re-balance the telescope. We do not necessarily need the whole process. We need the torques once the telescope is already balanced as a baseline.

May to Aug 2023 - M1M3 Surrogate and M1M3 Cell on the TMA

Nov 2023 to Jan 2024 - M1M3 Surrogate and Cell, M2 Surrogate and Cell on the TMA

Feb to Apr 2024 - M2 Surrogate and Cell on the TMA

## Setup notebook

Note that we have to `pip install DateTimeRange` in order to easily find the overlap between 2 time ranges

In [ ]:
# Notebook extensions for formatting and auto-reload libraries
%matplotlib inline
%load_ext lab_black
%load_ext autoreload
%autoreload 2

# Standard Python Libraries
import os
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd

from astropy.time import Time, TimeDelta
from collections import defaultdict
from datetime import datetime, timedelta
from scipy import stats
from datetimerange import DateTimeRange

# LSST Specific Libraries
from lsst_efd_client import EfdClient
from lsst.summit.utils.blockUtils import BlockParser
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.tmaUtils import (
    getCommandsDuringEvent,
    TMAEvent,
    TMAEventMaker,
    TMAState,
)

## Identify time periods with telemetry and where the TMA is still

In order to do that we will check the "lsst.sal.MTMount.logevent_elevationMotionState" and "lsst.sal.MTMount.logevent_azimuthMotionState" topics and select time ranges where
the `state` is equal to 1, meaning that the TMA is `stopped`

Once we have the time ranges in both azimuth and elevation we select the overlaps between both sets.

In [ ]:
async def get_time_range_by_axis(client, TMA_axis, begin, end):
    """
    Query the status of the TMA on a given axis (elevation or azimuth) during a time window and return
    a list of time ranges where the TMA  is still on this axis.

    Args:
        client: EFD client

        TMA_axis (str): Axis of the TMA to be checked ("elevation" or "azimuth").

        begin (Time) : Starting time

        end (Time) : End time

    Returns:
        time_range (list of DateTimeRange) : Range of times where the TMA is still on the considered TMA_axis
    """

    topic = f"lsst.sal.MTMount.logevent_{TMA_axis}MotionState"
    df_status = getEfdData(client, topic, begin=begin, end=end)

    select_stopped = df_status["state"] == 1

    # Select rows corresponding to motion state == stopped
    i_start = np.array(
        [
            df_status.index.get_loc(df_status[select_stopped].index[i])
            for i in range(len(df_status[select_stopped]))
        ]
    )
    t_start = list(df_status["state"].index[i_start])
    if i_start[-1] < len(df_status) - 1:
        t_end = list(df_status["state"].index[i_start + 1])
    else:
        t_end = list(df_status["state"].index[i_start[0:-1] + 1])
        t_end.append(pd.Timestamp(end_time.datetime64, tz="UTC"))

    time_ranges = []
    for i, (t1, t2) in enumerate(zip(t_start, t_end)):
        time_ranges.append(DateTimeRange(t1, t2))

    return time_ranges


def get_overlaps(t_range_1, t_range_2, min_delta):
    """
    Find the time overlaps between two lists of DateTimeRange. The overlap will only be considered if its duration is
    > min_delta seconds

    Args:
        t_range_1 (list of DateTimeRange)
        t_range_2 (list of DateTimeRange)
        min_delta (int) minimum duration in seconds of the time overlap

    Return:
        overlaps (list of DateTimeRange)
    """
    overlaps = []
    for r1 in t_range_1:
        for r2 in t_range_2:
            t_range_ok = r1.intersection(r2)
            if (t_range_ok.start_datetime != None) and (
                t_range_ok.timedelta.total_seconds() > min_delta
            ):
                overlaps.append(
                    DateTimeRange(t_range_ok.start_datetime, t_range_ok.end_datetime)
                )

    return overlaps

In [ ]:
# Create a directory to save plots
plot_dir = "./plots"
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

## Dates with interesting data

May to Aug 2023 - M1M3 Surrogate and M1M3 Cell on the TMA

* 2023-06-22

Nov 2023 to Jan 2024 - M1M3 Surrogate and Cell, M2 Surrogate and Cell on the TMA

* 2024-01-06 - Az: 59.6 deg
* 2024-01-09 - Az: 59.6 deg
* 2024-01-10 - Az: 60.3 deg
* 2024-01-12 - Az: 0 deg

Feb to Apr 2024 - M2 Surrogate and Cell on the TMA

 * 2024-03-28

In [ ]:
# Define the time period that we are going to investigate
date = "2023-06-22"
start_time = Time(f"{date} 00:00:00.00")
end_time = Time(f"{date} 23:59:59.00")

# Initialize EFD
client = EfdClient("usdf_efd")

In [ ]:
t_range_azi = await get_time_range_by_axis(client, "azimuth", start_time, end_time)
t_range_ele = await get_time_range_by_axis(client, "elevation", start_time, end_time)

min_delta = 10  # We want a minimum amount of data during the overlap period
overlaps = get_overlaps(t_range_azi, t_range_ele, min_delta)
print(
    f"We found {len(overlaps)} time periods where the TMA is still in both azimuth and elevation"
)

In [ ]:
overlaps

In [ ]:
# We are going to check the Torque as a function of the elevation angle for a given position of the TMA in azimuth
# So we will first identify what is the most common TMA position in azimuth and we will make the analysis for this
# position only

mean_pos_azi = []
std_pos_azi = []
for overlap in overlaps:
    df_azi = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        columns=columns,
        begin=Time(overlap.start_datetime),
        end=Time(overlap.end_datetime),
    )
    if len(df_azi) > 0:
        mean_pos_azi.append(np.mean(df_azi["actualPosition"]))
        std_pos_azi.append(np.std(df_azi["actualPosition"]))

# Find the most common TMA position in azimuth
hist = np.histogram(mean_pos_azi, bins=180)
imax = np.array(hist[0]).argmax()
az_min = hist[1][imax]
az_max = hist[1][imax + 1]
sel_azi = (az_min + az_max) / 2
print(
    f"The most common TMA azimuth angle is: {sel_azi:.1f} degrees - # of occurences: {hist[0][imax]}"
)

In [ ]:
columns = ["actualVelocity", "actualTorque", "actualPosition"]

cut_std_azi = 0.5  # Cut on the standard deviation of the azimuth position
cut_torque = 100  # We want the torque measurement to be stable for a given position in elevation. This is an absolute cut on the mean value (in N.m)
cut_mean_ele = 5  # We want to exclude measurement when the elevation angle is too small (cut in degrees)

mean_torque_ele = []
std_torque_ele = []
mean_pos_ele = []
std_pos_ele = []
mean_torque_azi = []
std_torque_azi = []
mean_pos_azi = []
std_pos_azi = []
mean_vel_azi = []
mean_vel_ele = []

for overlap in overlaps:
    df_ele = getEfdData(
        client,
        "lsst.sal.MTMount.elevation",
        columns=columns,
        begin=Time(overlap.start_datetime),
        end=Time(overlap.end_datetime),
    )
    df_azi = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        columns=columns,
        begin=Time(overlap.start_datetime),
        end=Time(overlap.end_datetime),
    )
    if (len(df_ele) == 0) or (len(df_azi) == 0):
        continue

    if (
        (np.mean(df_azi["actualPosition"]) > az_min)
        & (np.mean(df_azi["actualPosition"]) < az_max)
        & (np.mean(df_ele["actualPosition"]) > cut_mean_ele)
        & (np.std(df_azi["actualPosition"]) < cut_std_azi)
        & (np.std(df_ele["actualTorque"]) < cut_torque)
    ):

        mean_pos_azi.append(np.mean(df_azi["actualPosition"]))
        std_pos_azi.append(np.std(df_azi["actualPosition"]))
        mean_torque_ele.append(np.mean(df_ele["actualTorque"]))
        std_torque_ele.append(np.std(df_ele["actualTorque"]))
        mean_pos_ele.append(np.mean(df_ele["actualPosition"]))
        std_pos_ele.append(np.std(df_ele["actualPosition"]))
        mean_torque_azi.append(np.mean(df_azi["actualTorque"]))
        std_torque_azi.append(np.std(df_azi["actualTorque"]))
        mean_vel_ele.append(np.mean(df_ele["actualVelocity"]))

In [ ]:
fig, ax = plt.subplots(1, dpi=125, figsize=(10, 6))

ax.errorbar(
    mean_pos_ele,
    mean_torque_ele,
    xerr=std_pos_ele,
    yerr=std_torque_ele,
    fmt=".",
)
ax.set_xlabel("Elevation (degrees)")
ax.set_ylabel("Elevation torque (N.m)")
ax.set_xlim([0, 95])

fig.suptitle(f"{date} - Azimuth position: {sel_azi:.1f} degrees")
fig.tight_layout()
#plt.savefig(f"{plot_dir}/{date}-Az-{sel_azi:.1f}.png")
plt.show()

In [ ]:
_ = plt.hist(mean_pos_azi, bins=180)